# Importamos las Librerias



In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Cargamos los atributos 

Para este proyecto ocupamos la base  de datos MNIST 


In [2]:
##ocupamos los mismos valores que nos muestra el load data del mnist
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

#  Obtener los datos de entrada y normalizarlos



In [3]:
##Pre procesar las imagenes
##se divide entre 255 para normalizarlar la imagen al rango [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [4]:
##Expande las dimensiones de las imagenes a (28,28,1)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Visualizacion de los datos

In [6]:
##definimos nuestra funcion para que muestre los ejemplos
def mostrar_ejemplos(ejemplos, etiquetas):
    plt.figure(figsize=(10, 10))
##dentro de los valores de mnist mostrar 25 imagenes aleatorias como ejemplo
    for i in range(25):
        index = np.random.randint(0, ejemplos.shape[0]-1)
        img = ejemplos[index]
        label = etiquetas[index]
        ##graficar los valores 5, 5 
        plt.subplot(5, 5, i+1)
        plt.title(str(label))
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
    plt.show()


In [ ]:
mostrar_ejemplos(X_train,y_train)

# Modelo

In [8]:
##(Conv2D(unidades,tamano de kernel,)), el numero de unidades tiene que duplicarse
##28,28 es las dimensiones que maneja mnist en pixeles
##relu es nuestro rectificador de redes neuronales 
modelo = tf.keras.Sequential([
    ##capa de entrada, forma de 28*28 pixeles
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    ##operación matemática que combina dos funciones y genera una tercera funcion.
    ##el activador sera nuestro rectificador relu
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    ##para el pooling, o comparacion sobre entradas 
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    ##convierte el vector en probabilidades
    tf.keras.layers.GlobalAvgPool2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


In [14]:
##adam esta basado en la estimizacion adaptativa de momentos de primer y segundo orden
##sparse crea 2 variables total y recuento, se utilizan para calcular la frecuencia
##acuracy es nuestra precision
modelo.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', metrics='accuracy')
modelo.fit(X_train, y_train, batch_size=64, epochs=3, validation_split=0.2)
modelo.evaluate(X_test, y_test, batch_size=64)

Epoch 1/3
750/750 [==============================] - 421s 559ms/step - loss: 0.2202 - accuracy: 0.9407 - val_loss: 0.1032 - val_accuracy: 0.9712
Epoch 2/3
750/750 [==============================] - 388s 517ms/step - loss: 0.0524 - accuracy: 0.9835 - val_loss: 0.0926 - val_accuracy: 0.9727
Epoch 3/3
157/157 [==============================] - 26s 163ms/step - loss: 0.0911 - accuracy: 0.9702


[0.09108363837003708, 0.9702000021934509]

In [ ]:
##mandamos a llamar a nuestro modelo
modelo.summary()

# Aplicacion del modelo

In [16]:
import gradio as gr

In [17]:
##definimos nuestra funcion para predecir la imagen
def predecir_imagen(img):
    ##remodelamos
    img_3d=img.reshape(-1,28,28,1)
    img_norm=img_3d.astype('float32')/255
    prediction=modelo.predict(img_norm)
    pred=np.argmax(prediction)
    return pred


In [ ]:
##iface variable de la funcion interface
iface=gr.Interface(predecir_imagen,inputs="sketchpad",outputs="label")

In [ ]:
##lanzamos nuestra interface 
iface.launch(debug='True')